In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Cálculo do IQA da nossa amostra (Índice de Qualidade da Água)
data = pd.read_csv('nossos_dados.csv')
iqa_file = pd.ExcelFile('DadosIQA.xlsx')

# Relação dos arquivos de base para cálcudo do IQA com a nossa tabela criada
iqa_data = {
    'pH': {
        'name': 'pH',
        'lim_inf': [2, 2],
        'lim_sup': [12, 3],
        'q': [],
        'peso': 0.12
    },
    'Temperatura': {
        'name': 'Temperatura',
        'lim_inf': [-5, np.inf],
        'lim_sup': [15, 9],
        'q': [],
        'peso': 0.10
    },
    'Coliformes Termotolerantes': {
        'name': 'Coliformes',
        'lim_inf': [1, 100],
        'lim_sup': [1e5, 3],
        'q': [],
        'peso': 0.15
    },
    'Nitrogênio Total': {
        'name': 'N',
        'lim_inf': [0.5, 100],
        'lim_sup': [100, 1],
        'q': [],
        'peso': 0.10
    },
    'Turbidez': {
        'name': 'Turbidez',
        'lim_inf': [0.5, 100],
        'lim_sup': [100, 5],    
        'q': [],
        'peso': 0.08
    },
    'Oxigênio Dissolvido': {
        'name': 'OD',
        'lim_inf': [5, 5],
        'lim_sup': [140, 47],
        'q': [],
        'peso': 0.17
    },
    'Fósforo Total': {
        'name':'Fosforo',
        'lim_inf': [0.5, 100],
        'lim_sup': [10, 5],
        'q': [],
        'peso': 0.10
    },
    'Sólido Total': {
        'name': 'Residuos',
        'lim_inf': [0, 83],
        'lim_sup': [500, 32],
        'q': [],
        'peso': 0.08
    }
}

In [3]:
# Loop para calcular os Q's de cada parâmetro
for param in iqa_data.keys():
    data_per_param = data.loc[data['Parametro'] == param]
    iqa_df = pd.read_excel(iqa_file, iqa_data[param]['name'])

    for val in data_per_param['Valor']:
        if val < iqa_data[param]['lim_inf'][0]:
            q = iqa_data[param]['lim_inf'][1]
        elif val > iqa_data[param]['lim_sup'][0]:
            q = iqa_data[param]['lim_sup'][1]
        else:
            for index in range(1, len(iqa_df[iqa_df.columns[0]])):
                if val <= iqa_df[iqa_df.columns[0]][index]:
                    q = np.interp(val, 
                                    [float(iqa_df[iqa_df.columns[0]][index]), float(iqa_df[iqa_df.columns[0]][index-1])], 
                                    [iqa_df[iqa_df.columns[1]][index], iqa_df[iqa_df.columns[1]][index-1]])
                    break

        iqa_data[param]['q'].append(float(f'{q:.4f}'))

In [4]:
for param in iqa_data.keys():
    print(f'Valor de q para {param}: {iqa_data[f"{param}"]["q"]}')

Valor de q para pH: [69.3878]
Valor de q para Temperatura: [88.9796]
Valor de q para Coliformes Termotolerantes: [3.0]
Valor de q para Nitrogênio Total: [54.8602]
Valor de q para Turbidez: [44.9704]
Valor de q para Oxigênio Dissolvido: [29.6785]
Valor de q para Fósforo Total: [55.2]
Valor de q para Sólido Total: [32.0]


In [5]:
# Pesos dos parâmetros
pesos = np.array([iqa_data[param]['peso'] for param in iqa_data])

# Valores de qualidade (q) para cada amostra
q_values = np.array([iqa_data[param]['q'] for param in iqa_data])

IQA = np.prod(q_values**pesos[:, None], axis=0)
IQA = IQA * 2**0.1

print(f'IQA: {IQA[0]:.4f}')

IQA: 23.3687
